# Module 6 Activity

During this module we are learning about computational statistics which is a branch of mathematical sciences concerned with efficient methods for obtaining numerical solutions to statistically formulated problems.

In this activity you are going to go through the process of calculating Expectation Maximization as shown in the lecture notes and slides using the following two column example. 

In [5]:
import numpy as np

# Define the columns
column_1 = np.array([1, 4, 1, 4])
column_2 = np.array([2, 2, 3, 3])

# Create the 2-column array
x = np.column_stack((column_1, column_2))
print(x)

[[1 2]
 [4 2]
 [1 3]
 [4 3]]


In [16]:
k = 2

# Get column means and standard deviations by taking the mean and standard deviation of each column and transposing the result
column_mean = np.reshape(x.mean(axis=0), (2, 1))
print(f"Column mean is {column_mean}")

column_std_dev = np.reshape(x.std(axis=0, ddof=1), (2, 1))
print(f"Column std deviation is {column_std_dev}")

print(f"Shape of column mean: {column_mean.shape}, shape of column std dev: {column_std_dev.shape}")

# NOTE: Using example values from lecture notes in mu here for ease of verification 
# NOTE: We should be using np.random.randn(1, k) instead of [-0.1867, 0.7257]
# mu = (column_mean * np.array([1, 1])) + (column_std_dev * np.random.randn(1,k)).T

# Transpose mu as mu_k is a column major vector
mu = (column_mean * np.array([1, 1])) + (column_std_dev * np.array([-0.1867, 0.7257])).T
print(f"Mu is {mu}")

# Sigma
sigma = column_std_dev.mean() * np.array([1, 1])
print(f"Sigma is {sigma}")

# P_k
p_k = np.ones(x.shape[1]) / k
print(f"P_k is {p_k}")

print("Shapes:", x.shape, mu.shape, sigma.shape, p_k.shape)

Column mean is [[2.5]
 [2.5]]
Column std deviation is [[1.73205081]
 [0.57735027]]
Shape of column mean: (2, 1), shape of column std dev: (2, 1)
Mu is [[0.16635886 1.72211962]
 [1.23016023 2.07672008]]
Sigma is [1.15470054 1.15470054]
P_k is [0.5 0.5]
Shapes: (4, 2) (2, 2) (2,) (2,)


In [7]:
# Create E Step

def gaussian(x, mu, sigma):
    left_term = 1 / ((np.sqrt(2 * np.pi) * sigma) ** np.shape(x))
    right_term = np.exp(-0.5 * np.linalg.norm(x - mu) ** 2 / sigma ** 2)
    return left_term * right_term

gaussian_functions = []
gaussian_sums = []
for index in range(k):
    for feature in x:

        # I know I'm calculating the same thing twice here and for the sum, done for clarity 
        gaussian_functions.extend(p_k[index] * gaussian(feature, mu=mu[index], sigma=sigma[index]))

        sum = 0
        for ind in range(k):
            sum += p_k[ind] * gaussian(feature, mu=mu[ind], sigma=sigma[ind])
        gaussian_sums.extend(sum)

print(gaussian_functions)
print(gaussian_sums)

membership_probabilities = []
for function, sum in zip(gaussian_functions, gaussian_sums):
    membership_probabilities.append(function / sum)

print(membership_probabilities)

[0.054735249114340426, 0.005222216103570018, 0.042158772661716484, 0.004022311491464133, 0.026685400040000792, 0.02083020858323496, 0.025960937603389172, 0.020264704463277303]
[0.08142064915434122, 0.02605242468680498, 0.06811971026510566, 0.024287015954741437, 0.08142064915434122, 0.02605242468680498, 0.06811971026510566, 0.024287015954741437]
[0.6722526740186525, 0.20045029076372164, 0.6188924247863739, 0.1656157141313557, 0.3277473259813474, 0.7995497092362783, 0.3811075752136261, 0.8343842858686442]


In [8]:
# Create M Step
split_data = np.split(np.array(membership_probabilities), k, axis=0)

means = []
std_devs = []
probabilities = []

for membership_probability in split_data:
    top_sum = 0
    bottom_sum = 0
    membership_probabilities_sum = 0

    for index, feature in enumerate(x):
        top_sum += membership_probability[index] * feature
        membership_probabilities_sum += membership_probability[index] 

    mean_k = top_sum / membership_probabilities_sum
    means.append(mean_k)
    print(f"Mean_k is: {mean_k}")

    top_sum = 0
    for index, feature in enumerate(x):
        top_sum += membership_probability[index] * np.linalg.norm(feature - mean_k) ** 2

    std_dev_k = np.sqrt(0.5 * top_sum / membership_probabilities_sum)
    std_devs.append(std_dev_k)
    print(f"std_dev_k is: {std_dev_k}")

    probability = membership_probabilities_sum / x.shape[0]
    probabilities.append(probability)
    print(f"Probability is: {probability}")

print(f"Means: {means}")
print(f"Standard Deviations: {std_devs}")
print(f"Probabilities: {probabilities}")

Mean_k is: [1.66267841 2.47339059]
std_dev_k is: 0.948204748332092
Probability is: 0.41430277592502596
Mean_k is: [3.0922935  2.51882261]
std_dev_k is: 1.036540909983979
Probability is: 0.585697224074974
Means: [array([1.66267841, 2.47339059]), array([3.0922935 , 2.51882261])]
Standard Deviations: [0.948204748332092, 1.036540909983979]
Probabilities: [0.41430277592502596, 0.585697224074974]


In [9]:
# Create M-Step

Please add your Jupyter Notebook in HTML format to the discussion board found under Module 6. You are encouraged to review other students' submissions to check and discuss differences in your approaches. 